In [0]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [0]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import sklearn
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle
import numpy as np

In [0]:
def save_file(path,obj):
    pickle_out = open(path,"wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

In [0]:
#we don't need all descriptor to train Kmean or PCA,we can  just use some discriptors like 30 or 50 for each image
k_descriptors=30

#dataset's path 
dir_path_DataSet = 'drive/My Drive/medical_image_recognition/datasets/'

#fold name for saving files(kmean/PCA)
fold_name="k_means_PCA/"
save_fold='datanp_vgg/'

# docs=['miniMIT_Etus','chest_xray','kvasir-dataset-v2']#'kvasir-dataset-v2',]#,'chest_xray','kvasir-dataset-v2']#,'miniMIT_Etus','kvasir-dataset-v2']
docs=['miniMIT_Etus']
scales=[1,2]
layers=['fc1','fc2']

# N=1,2,3 ensemble

In [0]:
for doc in docs:
    for layer in layers:
        for n in scales:
            print(doc, n , layer)
            #laod_data, we dont need to load all data 
            X=np.load(dir_path_DataSet + doc + '/' + save_fold + doc +'X_train_'+layer + '(' + str(n) + ').npy')
            
            #this just for miniMIT bc we don't have many descriptorts 
            if doc=='miniMIT_Etus':
                if len(X.shape) == 4:
                  k_descriptors = X.shape[1]*X.shape[2]
                else:
                  k_descriptors = 1
                

            #reshape (d1,d2,d3,d4) -> (d1,d2*d3,d4)
            if len(X.shape) == 4:
              d=X.shape[1]*X.shape[2]
              X = np.reshape(X,(X.shape[0],d,X.shape[3]))
            else:
              d = 1
              X = np.reshape(X,(X.shape[0],d,X.shape[1]))
            
            #we take only some discriptors
            if doc!='miniMIT_Etus':
                #take random descriptors
                indexes = np.arange(d)
                np.random.shuffle(indexes)
                indexes=indexes[:k_descriptors]
                X=X[:,indexes,:]
                
            
        
            if(n==0):
                x_all=X
            else:
                x_all=np.concatenate((x_all,X),axis=1)
                del X
            #x_test_all=np.concatenate((x_test_all,x_test),axis=1)

            print(x_all.shape)
            
        x_all=np.reshape(x_all,(x_all.shape[0]*x_all.shape[1],x_all.shape[2]))
        print(x_all.shape)
        #normalization l2
        x_all = sklearn.preprocessing.normalize(x_all, norm = 'l2', axis = 1)
        #centralisation
        x_all = StandardScaler(with_std=False).fit_transform(x_all)
        
        #PCA 
        pca = PCA(n_components=128)
        x_all = sklearn.preprocessing.normalize(x_all, norm = 'l2', axis = 1)
        pca.fit(x_all)


        #SAVE the PCA  pickle
        save_file(dir_path_DataSet+doc + '/' + fold_name+doc+'_'+layer+'_'+'PCA.pickle',pca)

        #transformation with PCA
        print("PCA fit ...")
        x_train=pca.transform(x_all)
        del x_all

        #normalisation
        x_train = sklearn.preprocessing.normalize(x_train, norm = 'l2', axis = 1)
        print("pca shape")
        print(x_train.shape)

        #kmeans 64 clusters
        print("Kmean1 64 ...")
        kmeans1 = KMeans(n_clusters=64, random_state=0).fit(x_train)

        #SAVE the kmeans1  pickle
        save_file(dir_path_DataSet+doc + '/' + fold_name+doc+'_'+layer+'_'+'kmean1.pickle',kmeans1)

        print("Kmean2 256 ...")
        #kmeans 256 clusters
        kmeans2 = KMeans(n_clusters=256, random_state=0).fit(x_train)

        #SAVE the kmeans2  pickle
        save_file(dir_path_DataSet+doc + '/' + fold_name+doc+'_'+layer+'_'+'kmean2.pickle',kmeans2)
        print('end of ',doc , n , layer)
        
print("END")
      

miniMIT_Etus 0 fc2
(120, 1, 4096)
miniMIT_Etus 1 fc2
(120, 122, 4096)
miniMIT_Etus 2 fc2
(120, 563, 4096)
(67560, 4096)


/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


PCA fit ...
pca shape
(67560, 128)
Kmean1 64 ...
Kmean2 256 ...
end of  miniMIT_Etus 2 fc2
END


# N=1,2,3 séparément

In [14]:
for doc in docs:
    for layer in layers:
        for n in scales:
            print(doc, n , layer)
            #laod_data, we dont need to load all data 
            X=np.load(dir_path_DataSet + doc + '/' + save_fold + doc +'X_train_'+layer + '(' + str(n) + ').npy')
            
            #this just for miniMIT bc we don't have many descriptorts 
            if doc=='miniMIT_Etus':
                if len(X.shape) == 4:
                  k_descriptors = X.shape[1]*X.shape[2]
                else:
                  k_descriptors = 1
                

            #reshape (d1,d2,d3,d4) -> (d1,d2*d3,d4)
            if len(X.shape) == 4:
              d=X.shape[1]*X.shape[2]
              X = np.reshape(X,(X.shape[0],d,X.shape[3]))
            else:
              d = 1
              X = np.reshape(X,(X.shape[0],d,X.shape[1]))
            
            #we take only some discriptors
            if doc!='miniMIT_Etus':
                #take random descriptors
                indexes = np.arange(d)
                np.random.shuffle(indexes)
                indexes=indexes[:k_descriptors]
                X=X[:,indexes,:]
                
            x_all=X
            x_all=np.reshape(x_all,(x_all.shape[0]*x_all.shape[1],x_all.shape[2]))
            print(x_all.shape)
            #normalization l2
            x_all = sklearn.preprocessing.normalize(x_all, norm = 'l2', axis = 1)
            #centralisation
            x_all = StandardScaler(with_std=False).fit_transform(x_all)

            #PCA 
            pca = PCA(n_components=128)
            x_all = sklearn.preprocessing.normalize(x_all, norm = 'l2', axis = 1)
            pca.fit(x_all)


            #SAVE the PCA  pickle
            save_file(dir_path_DataSet+doc + '/' + fold_name+doc+'_'+layer+'_'+'n'+str(n)+'_PCA.pickle',pca)

            #transformation with PCA
            print("PCA fit ...")
            x_train=pca.transform(x_all)
            del x_all

            #normalisation
            x_train = sklearn.preprocessing.normalize(x_train, norm = 'l2', axis = 1)
            print("pca shape")
            print(x_train.shape)

            #kmeans 64 clusters
            print("n = {}, Kmean1 64 ...".format(n))
            kmeans1 = KMeans(n_clusters=64, random_state=0).fit(x_train)

            #SAVE the kmeans1  pickle
            save_file(dir_path_DataSet+doc + '/' + fold_name+doc+'_'+layer+'_'+'n'+str(n)+'_kmean1.pickle',kmeans1)

            print("n = {}, Kmean1 256 ...".format(n))
            #kmeans 256 clusters
            kmeans2 = KMeans(n_clusters=256, random_state=0).fit(x_train)

            #SAVE the kmeans2  pickle
            save_file(dir_path_DataSet+doc + '/' + fold_name+doc+'_'+layer+'_'+'n'+str(n)+'_kmean2.pickle',kmeans2)
            print('end of ',doc , n , layer)
        
print("END")
      

miniMIT_Etus 0 fc1
(120, 4096)
PCA fit ...
pca shape
(120, 64)
n = 0, Kmean1 64 ...


/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (31) found smaller than n_clusters (64). Possibly due to duplicate points in X.
  return_n_iter=True)


end of  miniMIT_Etus 0 fc1
miniMIT_Etus 0 fc2
(120, 4096)
PCA fit ...
pca shape
(120, 64)
n = 0, Kmean1 64 ...


/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


end of  miniMIT_Etus 0 fc2
END


In [12]:
# x_all.shape

(120, 4096)